In [12]:
import pandas as pd
df=pd.read_csv('main.csv')


In [13]:
df.head()

,title,overview,genre_list,crew_list,cast_list,original_language,comp_list,comb
0,Kingsman: The Golden Circle,After the Kingsman's headquarters are destroye...,Action Adventure Comedy,Matthew Vaughn,Taron Egerton Colin Firth Mark Strong Channing...,en,unknown,Action Adventure Comedy Taron Egerton Colin Fi...
1,Blade Runner 2049,Young Blade Runner K's discovery of a long-bur...,Action Drama Mystery,Denis Villeneuve,Harrison Ford Ryan Gosling Ana de Armas Dave B...,en,unknown,Action Drama Mystery Harrison Ford Ryan Goslin...
2,Dunkirk,"Allied soldiers from Belgium, the British Empi...",Action Drama History,Christopher Nolan,Fionn Whitehead Barry Keoghan Mark Rylance Tom...,en,unknown,Action Drama History Fionn Whitehead Barry Keo...
3,Thor: Ragnarok,"Imprisoned on the planet Sakaar, Thor must rac...",Action Adventure Comedy,Taika Waititi,Chris Hemsworth Tom Hiddleston Cate Blanchett ...,en,unknown,Action Adventure Comedy Chris Hemsworth Tom Hi...
4,It,"In the summer of 1989, a group of bullied kids...",Horror,Andy Muschietti,Bill Skarsgård Jaeden Martell Finn Wolfhard So...,en,unknown,Horror Bill Skarsgård Jaeden Martell Finn Wolf...


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


df_comb =df

In [7]:
cv=CountVectorizer()
count_matrix = cv.fit_transform(df_comb['comb'])

In [8]:
similarity =cosine_similarity(count_matrix)


In [9]:

def recommend(movie):
  i=df_comb.loc[df_comb['title']==movie].index[0]
  List =list(enumerate(similarity[i]))
  List_sorted =sorted(List,reverse=True,key=lambda x:x[1])
  Final_list=List_sorted[1:15]        # 1:15 because 0 will be the movie itself because of 1 correlation.
  recommend_list =[]
  for i in range(len(Final_list)):
    a=Final_list[i][0]
    recommend_list.append(df_comb['title'][a])

  return recommend_list

In [10]:
my_movies = recommend("Superman Returns")
for m in my_movies:
  print(m)

X-Men: Days of Future Past
X-Men
X2
Dylan Dog: Dead of Night
Fantastic Four
Divergent
Beyond the Sea
Avatar
Man of Steel
Jupiter Ascending
The Wolverine
Superman
Superman II
Underworld: Evolution


In [32]:
my_movies = recommend("Wonder Woman")
for m in my_movies:
  print(m)

Batman v Superman: Dawn of Justice
Justice League
Star Trek Into Darkness
Thor: The Dark World
Thor
Star Trek
Vertical Limit
Bohemian Rhapsody
Star Trek Beyond
Fantastic 4: Rise of the Silver Surfer
Cirque du Freak: The Vampire's Assistant
Jurassic World: Fallen Kingdom
Puss in Boots
The Princess Bride


In [33]:
df.to_csv('final_movie_data.csv',index=False)